In [3]:
import sqlite3
import pandas as pd
from itertools import permutations

import blpapi
from blpapi import Session, SessionOptions
import datetime
from datetime import timedelta

In [2]:
def init_session():
    session = Session()
    if not session.start():
        print("Failed to start session")
        return
    if not session.openService("//blp/refdata"):
        print("Failed to open service.")
        return
    return session


In [10]:
def get_price_data(tick, session):
    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("HistoricalDataRequest")

    start_date = (datetime.datetime.today() - datetime.timedelta(days=365*10)).strftime("%Y%m%d")
    end_date = datetime.datetime.today().strftime("%Y%m%d")

    request.getElement("securities").appendValue(tick)
    request.getElement("fields").appendValue('PX_LAST')
    request.set("startDate", start_date)
    request.set("endDate", end_date)
    request.set("periodicitySelection", "DAILY")

    session.sendRequest(request)

    dates = []
    values = []

    while True:
        event = session.nextEvent()
        for msg in event:
            print(msg)
            if msg.hasElement("securityData"):
                data = msg.getElement("securityData").getElement("fieldData")
                for i in range(data.numValues()):
                    item = data.getValueAsElement(i)
                    if item.hasElement('PX_LAST'):
                        dates.append(item.getElementAsDatetime("date"))
                        values.append(item.getElementAsFloat('PX_LAST'))
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    session.stop()
    return pd.Series(values, index=pd.to_datetime(dates), name=tick)



In [11]:
session = init_session()
data = get_price_data('AAPL US EQUITY', session)

SessionConnectionUp = {
    server = "127.0.0.1:8194"
    serverId = "bbcomm-BLOOM-LABPC16-114164"
    encryptionStatus = "Clear"
    compressionStatus = "Uncompressed"
}

SessionStarted = {
    initialEndpoints[] = {
        initialEndpoints = {
            address = "127.0.0.1:8194"
        }
    }
}

CID: {[ valueType=AUTOGEN classId=0 value=10 ]}
RequestId: 5ef8dd0f-ae00-5963-0000-793761810600
ServiceOpened = {
    serviceName = "//blp/refdata"
}

CID: {[ valueType=AUTOGEN classId=0 value=12 ]}
RequestId: 46618171-797f-4d6c-a2ef-b296403ec8b0
HistoricalDataResponse = {
    securityData = {
        security = "AAPL US EQUITY"
        eidData[] = {
        }
        sequenceNumber = 0
        fieldExceptions[] = {
        }
        fieldData[] = {
            fieldData = {
                date = 2015-06-16
                PX_LAST = 31.900000
            }
            fieldData = {
                date = 2015-06-17
                PX_LAST = 31.825000
            }
            fieldData

In [12]:
data

2015-06-16     31.9000
2015-06-17     31.8250
2015-06-18     31.9700
2015-06-19     31.6500
2015-06-22     31.9025
                ...   
2025-06-09    201.4500
2025-06-10    202.6700
2025-06-11    198.7800
2025-06-12    199.2000
2025-06-13    196.4100
Name: AAPL US EQUITY, Length: 2515, dtype: float64

In [14]:

TICKER_PATH = r"data\sp500_tickers.csv"
SAVE_PATH = r"data\sp500_earnings_dates.csv"
FIELD = "DY895"
START_DATE = (datetime.datetime.now() - datetime.timedelta(days=365 * 10)).strftime("%Y%m%d")
END_DATE = datetime.datetime.now().strftime("%Y%m%d")

def init_session():
    options = SessionOptions()
    options.setServerHost("localhost")
    options.setServerPort(8194)
    session = Session(options)
    if not session.start():
        raise RuntimeError("Failed to start session.")
    if not session.openService("//blp/refdata"):
        raise RuntimeError("Failed to open //blp/refdata.")
    return session

def get_earnings_data(tickers, session):
    refDataService = session.getService("//blp/refdata")
    data = {}

    for ticker in tickers:
        request = refDataService.createRequest("HistoricalDataRequest")
        request.getElement("securities").appendValue(ticker)
        request.getElement("fields").appendValue(FIELD)
        request.set("startDate", START_DATE)
        request.set("endDate", END_DATE)
        request.set("periodicitySelection", "DAILY")

        session.sendRequest(request)

        dates = []

        while True:
            event = session.nextEvent()
            for msg in event:
                if msg.hasElement("securityData"):
                    sec_data = msg.getElement("securityData")
                    field_data = sec_data.getElement("fieldData")
                    for i in range(field_data.numValues()):
                        item = field_data.getValueAsElement(i)
                        if item.hasElement(FIELD):
                            date_val = item.getElementAsDatetime(FIELD)
                            dates.append(date_val.date() if hasattr(date_val, 'date') else date_val)
            if event.eventType() == blpapi.Event.RESPONSE:
                break

        data[ticker] = dates

    # Convert to DataFrame with ragged columns
    df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
    return df

def main():
    tickers_df = pd.read_csv(TICKER_PATH, header=None)
    tickers = tickers_df[0].tolist()
    session = init_session()
    print("Fetching earnings data...")
    df = get_earnings_data(tickers[:5], session)
    print("Saving to CSV...")
    df.to_csv(SAVE_PATH, index=False)
    print(f"Saved to {SAVE_PATH}")

if __name__ == "__main__":
    main()



Fetching earnings data...
Saving to CSV...
Saved to data\sp500_earnings_dates.csv


In [57]:
def init_session():
    options = SessionOptions()
    options.setServerHost("localhost")
    options.setServerPort(8194)
    session = Session(options)
    if not session.start():
        raise RuntimeError("Failed to start session.")
    if not session.openService("//blp/refdata"):
        raise RuntimeError("Failed to open //blp/refdata.")
    return session

def get_earnings_dates(ticker, session):
    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("ReferenceDataRequest")
    request.append("securities", ticker)
    request.append("fields", "DY895")  # Earnings announcement dates

    session.sendRequest(request)

    earnings_dates = []
    ten_years_ago = datetime.date.today() - timedelta(days=365 * 10)

    while True:
        event = session.nextEvent()
        for msg in event:
            if msg.hasElement("securityData"):
                #print(msg)
                securityDataArray = msg.getElement("securityData")
                for i in range(securityDataArray.numValues()):
                    securityData = securityDataArray.getValueAsElement(i)
                    fieldData = securityData.getElement("fieldData")
                    if fieldData.hasElement("DY895"):
                        dy895Data = fieldData.getElement("DY895")
                        # DY895 can be an array of dates
                        for j in range(dy895Data.numValues()):
                            data = dy895Data.getValueAsElement(j)
                            date = data['Announcement Date']
                            if date < ten_years_ago:
                                break
                            earnings_dates.append(date)
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    earnings_dates.reverse()
    return earnings_dates

if __name__ == "__main__":
    session = init_session()

    # Load tickers from CSV
    ticker_df = pd.read_csv("data/sp500_tickers.csv", header=None)
    tickers = ticker_df[0].tolist()

    all_data = {}
    for i, ticker in enumerate(tickers[:3]):
        try:
            print(f"[{i+1}/{len(tickers)}] Getting earnings for {ticker}")
            dates = get_earnings_dates(f'{ticker} US EQUITY', session)
            all_data[ticker] = dates
        except Exception as e:
            print(f"Failed to get data for {ticker}: {e}")
            all_data[ticker] = []

    # Convert to DataFrame (columns = tickers, rows = earnings dates)
    df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in all_data.items()]))
    df.to_csv('data/earnings_dates.csv')


[1/503] Getting earnings for MMM
[2/503] Getting earnings for AOS
[3/503] Getting earnings for ABT


In [56]:
get_earnings_dates('MMM US EQUITY', session=init_session())

[datetime.date(2015, 7, 23),
 datetime.date(2015, 10, 22),
 datetime.date(2016, 1, 26),
 datetime.date(2016, 4, 26),
 datetime.date(2016, 7, 26),
 datetime.date(2016, 10, 25),
 datetime.date(2017, 1, 24),
 datetime.date(2017, 4, 25),
 datetime.date(2017, 7, 25),
 datetime.date(2017, 10, 24),
 datetime.date(2018, 1, 25),
 datetime.date(2018, 4, 24),
 datetime.date(2018, 7, 24),
 datetime.date(2018, 10, 23),
 datetime.date(2019, 1, 29),
 datetime.date(2019, 4, 25),
 datetime.date(2019, 7, 25),
 datetime.date(2019, 10, 24),
 datetime.date(2020, 1, 28),
 datetime.date(2020, 4, 28),
 datetime.date(2020, 7, 28),
 datetime.date(2020, 10, 27),
 datetime.date(2021, 1, 26),
 datetime.date(2021, 4, 27),
 datetime.date(2021, 7, 27),
 datetime.date(2021, 10, 26),
 datetime.date(2022, 1, 25),
 datetime.date(2022, 4, 26),
 datetime.date(2022, 7, 26),
 datetime.date(2022, 10, 25),
 datetime.date(2023, 1, 24),
 datetime.date(2023, 4, 25),
 datetime.date(2023, 7, 25),
 datetime.date(2023, 10, 24),
 date

In [4]:


def init_session():
    options = SessionOptions()
    options.setServerHost("localhost")
    options.setServerPort(8194)
    session = Session(options)
    if not session.start():
        raise RuntimeError("Failed to start session.")
    if not session.openService("//blp/refdata"):
        raise RuntimeError("Failed to open //blp/refdata.")
    return session

def get_mkt_cap(ticker, session):
    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("HistoricalDataRequest")
    request.append("securities", ticker)
    request.append("fields", "CUR_MKT_CAP")  # Earnings announcement dates

    end_date = datetime.datetime.now().strftime('%Y%m%d')
    start_date = (datetime.datetime.now()-timedelta(365*10)).strftime('%Y%m%d')

    request.set("startDate", start_date)
    request.set("endDate", end_date)

    session.sendRequest(request)

    values = {}
    while True:
        event = session.nextEvent()
        for msg in event:
            if msg.hasElement("securityData"):
                #print(msg)
                securityData = msg.getElement("securityData")
                fieldData = securityData.getElement("fieldData")
                for i in range(fieldData.numValues()):
                    data = fieldData.getValue(i)
                    values[data['date']] = data['CUR_MKT_CAP']
                        
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    return values

In [5]:
session = init_session()
values = get_mkt_cap('AAPL US EQUITY', session)

In [6]:
values

{datetime.date(2015, 6, 16): 735107.4006,
 datetime.date(2015, 6, 17): 733379.0916,
 datetime.date(2015, 6, 18): 736720.4889,
 datetime.date(2015, 6, 19): 729346.3708,
 datetime.date(2015, 6, 22): 735165.0109,
 datetime.date(2015, 6, 23): 731823.6136,
 datetime.date(2015, 6, 24): 738045.5258,
 datetime.date(2015, 6, 25): 734531.2976,
 datetime.date(2015, 6, 26): 730210.5252,
 datetime.date(2015, 6, 29): 717421.0391,
 datetime.date(2015, 6, 30): 722577.1608,
 datetime.date(2015, 7, 1): 729346.3708,
 datetime.date(2015, 7, 2): 728424.606,
 datetime.date(2015, 7, 6): 725889.7529,
 datetime.date(2015, 7, 7): 724103.8337,
 datetime.date(2015, 7, 8): 706129.4207,
 datetime.date(2015, 7, 9): 691726.8463,
 datetime.date(2015, 7, 10): 710219.7519,
 datetime.date(2015, 7, 13): 723931.0028,
 datetime.date(2015, 7, 14): 723642.9513,
 datetime.date(2015, 7, 15): 730613.7973,
 datetime.date(2015, 7, 16): 740349.9377,
 datetime.date(2015, 7, 17): 746744.6807,
 datetime.date(2015, 7, 20): 760859.2037,

In [8]:
from tqdm import tqdm
if __name__ == "__main__":
    session = init_session()

    # Load tickers from CSV
    ticker_df = pd.read_csv("data/sp500_tickers.csv", header=None)
    tickers = ticker_df[0].tolist()

    all_data = {}
    for i, ticker in tqdm(enumerate(tickers)):
        try:
            dates = get_mkt_cap(f'{ticker} US EQUITY', session)
            all_data[ticker] = dates
        except Exception as e:
            print(f"Failed to get data for {ticker}: {e}")
            all_data[ticker] = []

    # Convert to DataFrame (columns = tickers, rows = earninbgs dates)
    df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in all_data.items()]))
    df.to_csv('data/cur_mkt_cap.csv')
    print('Data saved to data/cur_mkt_cap.csv')

503it [03:23,  2.47it/s]


Data saved to data/cur_mkt_cap.csv


In [1]:
import blpapi
from blpapi import Session, SessionOptions
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
from tqdm import tqdm

def init_session():
    options = SessionOptions()
    options.setServerHost("localhost")
    options.setServerPort(8194)
    session = Session(options)
    if not session.start():
        raise RuntimeError("Failed to start session.")
    if not session.openService("//blp/refdata"):
        raise RuntimeError("Failed to open //blp/refdata.")
    return session

def get_tot_ret(ticker, session):
    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("HistoricalDataRequest")
    request.append("securities", ticker)
    request.append("fields", "RT116")  

    end_date = datetime.datetime.now().strftime('%Y%m%d')
    start_date = (datetime.datetime.now()-timedelta(365*10)).strftime('%Y%m%d')

    request.set("startDate", start_date)
    request.set("endDate", end_date)

    session.sendRequest(request)

    values = {}
    while True:
        event = session.nextEvent()
        for msg in event:
            if msg.hasElement("securityData"):
                #print(msg)
                securityData = msg.getElement("securityData")
                fieldData = securityData.getElement("fieldData")
                for i in range(fieldData.numValues()):
                    data = fieldData.getValue(i)
                    values[data['date']] = data['RT116']
                        
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    return values

from tqdm import tqdm
if __name__ == "__main__":
    session = init_session()

    # Load tickers from CSV
    ticker_df = pd.read_csv("data/sp500_tickers.csv", header=None)
    tickers = ticker_df[0].tolist()

    all_data = {}
    for i, ticker in tqdm(enumerate(tickers[:5])):
        try:
            dates = get_tot_ret(f'{ticker} US EQUITY', session)
            all_data[ticker] = dates
        except Exception as e:
            print(f"Failed to get data for {ticker}: {e}")
            all_data[ticker] = []

    # Convert to DataFrame (columns = tickers, rows = earninbgs dates)
    df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in all_data.items()]))
    df.to_csv('data/bbg_tot_ret.csv')
    print('Data saved to data/bbg_tot_ret.csv')


ImportError: DLL load failed while importing hashtable: The paging file is too small for this operation to complete.

In [1]:
import blpapi
from blpapi import Session, SessionOptions
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
from tqdm import tqdm

def init_session():
    options = SessionOptions()
    options.setServerHost("localhost")
    options.setServerPort(8194)
    session = Session(options)
    if not session.start():
        raise RuntimeError("Failed to start session.")
    if not session.openService("//blp/refdata"):
        raise RuntimeError("Failed to open //blp/refdata.")
    return session

def get_tot_ret(ticker, session):
    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("HistoricalDataRequest")
    request.append("securities", ticker)
    request.append("fields", "RT116")  

    end_date = datetime.datetime.now().strftime('%Y%m%d')
    start_date = (datetime.datetime.now()-timedelta(365*10)).strftime('%Y%m%d')

    request.set("startDate", start_date)
    request.set("endDate", end_date)

    session.sendRequest(request)

    values = {}
    while True:
        event = session.nextEvent()
        for msg in event:
            if msg.hasElement("securityData"):
                #print(msg)
                securityData = msg.getElement("securityData")
                fieldData = securityData.getElement("fieldData")
                for i in range(fieldData.numValues()):
                    data = fieldData.getValue(i)
                    values[data['date']] = data['RT116']
                        
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    return values


session = init_session()
val = get_tot_ret('AAPL US EQUITY', session)

In [2]:
val

{datetime.date(2015, 6, 16): 31.9,
 datetime.date(2015, 6, 17): 31.825,
 datetime.date(2015, 6, 18): 31.97,
 datetime.date(2015, 6, 19): 31.65,
 datetime.date(2015, 6, 22): 31.9025,
 datetime.date(2015, 6, 23): 31.7575,
 datetime.date(2015, 6, 24): 32.0275,
 datetime.date(2015, 6, 25): 31.875,
 datetime.date(2015, 6, 26): 31.6875,
 datetime.date(2015, 6, 29): 31.1325,
 datetime.date(2015, 6, 30): 31.3563,
 datetime.date(2015, 7, 1): 31.65,
 datetime.date(2015, 7, 2): 31.61,
 datetime.date(2015, 7, 6): 31.5,
 datetime.date(2015, 7, 7): 31.4225,
 datetime.date(2015, 7, 8): 30.6425,
 datetime.date(2015, 7, 9): 30.0175,
 datetime.date(2015, 7, 10): 30.82,
 datetime.date(2015, 7, 13): 31.415,
 datetime.date(2015, 7, 14): 31.4025,
 datetime.date(2015, 7, 15): 31.705,
 datetime.date(2015, 7, 16): 32.1275,
 datetime.date(2015, 7, 17): 32.405,
 datetime.date(2015, 7, 20): 33.0175,
 datetime.date(2015, 7, 21): 32.6875,
 datetime.date(2015, 7, 22): 31.305,
 datetime.date(2015, 7, 23): 31.29,
 dat

In [3]:
options = blpapi.SessionOptions()
print(options.serverHost())  # usually "localhost"


127.0.0.1
